In [8]:
import pandas as pd
import streamlit as st

# ------------------------------
# 📥 Carregar os dataframes
# ------------------------------
df_localidades = pd.read_csv("localidades.csv")
df_acidentes = pd.read_csv("acidentes.csv", low_memory=False)


In [2]:

# ------------------------------
# 🔍 Filtrar apenas para RO
# ------------------------------
df_localidades = df_localidades[df_localidades['uf'] == 'RO']
df_acidentes = df_acidentes[df_acidentes['uf_acidente'] == 'RO']

# ------------------------------
# 📊 Agrupar acidentes por município (IBGE)
# ------------------------------
df_acidentes_por_cidade = df_acidentes.groupby('codigo_ibge').size().reset_index(name='total_acidentes')

# Usar apenas uma linha única por cidade para pegar o nome
municipios_unicos = df_localidades[['codigo_ibge', 'municipio']].drop_duplicates(subset='codigo_ibge')

df_acidentes_por_cidade = pd.merge(
    df_acidentes_por_cidade,
    municipios_unicos,
    on='codigo_ibge',
    how='left'
)

df_acidentes_por_cidade.sort_values(by='total_acidentes', ascending=False, inplace=True)

In [3]:
df_acidentes_por_cidade

,codigo_ibge,total_acidentes,municipio
16,1100205,5592,PORTO VELHO
11,1100122,1374,JI-PARANA
21,1100304,1330,VILHENA
3,1100049,971,CACOAL
1,1100023,966,ARIQUEMES
19,1100288,696,ROLIM DE MOURA
10,1100114,449,JARU
15,1100189,405,PIMENTA BUENO
8,1100098,266,ESPIGAO D OESTE
14,1100155,263,OURO PRETO DO OESTE


In [4]:
# Obter população única por município
populacao_unica = df_localidades[['codigo_ibge', 'municipio', 'qtde_habitantes']].drop_duplicates(subset='codigo_ibge')

# Juntar população com acidentes
df_relativo = pd.merge(df_acidentes_por_cidade, populacao_unica, on=['codigo_ibge', 'municipio'], how='left')

# Remover municípios com população inválida
df_relativo = df_relativo[df_relativo['qtde_habitantes'] > 0].copy()

# Calcular acidentes por mil habitantes
df_relativo['acidentes_por_mil_hab'] = (df_relativo['total_acidentes'] / df_relativo['qtde_habitantes']) * 1000

# Selecionar top 5 cidades com maior índice relativo
top5_relativo = df_relativo.sort_values(by='acidentes_por_mil_hab', ascending=False).head(5)

top5_relativo


,codigo_ibge,total_acidentes,municipio,qtde_habitantes,acidentes_por_mil_hab
2,1100304,1330,VILHENA,104517,12.725203
5,1100288,696,ROLIM DE MOURA,55748,12.484753
3,1100049,971,CACOAL,86416,11.236345
7,1100189,405,PIMENTA BUENO,37098,10.917031
1,1100122,1374,JI-PARANA,131026,10.486468


In [5]:
# -----------------------------------
# 🚘 Gráfico: Acidentes por mil veículos
# -----------------------------------

# Obter frota total por município (único)
frota_unica = df_localidades[['codigo_ibge', 'municipio', 'frota_total']].drop_duplicates(subset='codigo_ibge')

# Juntar com os dados de acidentes
df_veiculos = pd.merge(df_acidentes_por_cidade, frota_unica, on=['codigo_ibge', 'municipio'], how='left')

# Filtrar municípios com frota válida (> 0)
df_veiculos = df_veiculos[df_veiculos['frota_total'] > 0].copy()

# Calcular acidentes por mil veículos
df_veiculos['acidentes_por_mil_veic'] = (df_veiculos['total_acidentes'] / df_veiculos['frota_total']) * 1000

# Selecionar Top 5 cidades com maior índice relativo
top5_veiculos = df_veiculos.sort_values(by='acidentes_por_mil_veic', ascending=False).head(5)
top5_veiculos

,codigo_ibge,total_acidentes,municipio,frota_total,acidentes_por_mil_veic
0,1100205,5592,PORTO VELHO,302563,18.482101
2,1100304,1330,VILHENA,72965,18.227917
14,1100809,143,CANDEIAS DO JAMARI,7994,17.888416
25,1101104,71,ITAPUA DO OESTE,3986,17.812343
7,1100189,405,PIMENTA BUENO,27043,14.976149


In [ ]:
''

,Unnamed: 0,chv_localidade,ano_referencia,mes_referencia,mes_ano_referencia,regiao,uf,codigo_ibge,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante
46464,234864,RO1101609202201,2022,1,12022,NORTE,RO,1101609,THEOBROMA,nao,10348,4417,3508
46465,234865,RO1101484202201,2022,1,12022,NORTE,RO,1101484,SAO FELIPE D OESTE,nao,4962,3368,2675
46466,234866,RO0000000202201,2022,1,12022,NORTE,RO,0,NAO INFORMADO,nao,0,0,0
46467,234867,RO1100700202201,2022,1,12022,NORTE,RO,1100700,CAMPO NOVO DE RONDONIA,nao,14391,3535,2672
46468,234868,RO1101005202201,2022,1,12022,NORTE,RO,1101005,GOVERNADOR JORGE TEIXEIRA,nao,7130,4747,3748
